In [39]:
import bisect
from collections import defaultdict
import gzip
import io
import os

import pandas as pd
from tqdm.notebook import tqdm

In [17]:
wd = '~'
hd = os.getcwd() + '/'
hg38_gtf = f'{"/".join(hd.split("/")[0:-2])}/hg38.ncbiRefSeq.gtf.gz'

In [15]:
bed_files = []

for head, directory, files in os.walk(wd):
    for file in files:
        if file.endswith('.bed'):
            bed_files.append(f'{head}/{file}')

In [48]:
def binary_search(site, feature_spans):
    search_features = [x[0] for x in feature_spans[site[0]]]
    bisect_index = bisect.bisect(search_features, site[1])
    upstream_gene = feature_spans[site[0]][bisect_index - 1]
    upstream_dist = upstream_gene[0] - site[1]
    try:
        downstream_gene = feature_spans[site[0]][bisect_index]
    except IndexError as e:
        downstream_gene = 'NA'
        downstream_dist = 'NA'
        bookmark = upstream_gene
    else:
        downstream_dist = downstream_gene[0] - site[1]
        bookmark = upstream_gene if abs(upstream_dist) < downstream_dist else downstream_gene
    return dict(upstream_gene_id=upstream_gene[1], upstream_gene_start=upstream_gene[0], feature_upstream_dist=upstream_dist,
                downstream_gene_id=downstream_gene[1], downstream_gene_start=downstream_gene[0], feature_downstream_dist=downstream_dist,
                bookmark_gene=bookmark[1])

In [19]:
def get_book_mark_info(site, bookmark_gene):
    if bookmark_gene[0]['start'] <= site[1] <= bookmark_gene[0]['end']:
        feature_hits = []
        for exon in bookmark_gene[1:]:
            if exon['start'] <= site[1] <= exon['end']:
                if exon['feature'] != 'transcript':
                    feature_hits.append(exon['feature'])
        if feature_hits:
            if 'exon' in feature_hits:
                return 'exon'
            else:
                return feature_hits[0]
        return 'intragenic'
    else:
        return 'intergenic'

In [20]:
def update_coefs_meta(coefs, gene_reference, feature_spans):
    for site in coefs:
        chrom, start, end = site.split(':')
        bookmark_info = binary_search((chrom, int(pos)), feature_spans)
        bookmark_info['hit_type'] = get_book_mark_info((chrom, int(pos)), gene_reference[bookmark_info['bookmark_gene']])
        bookmark_info['chrom'] = chrom
        bookmark_info['pos'] = pos
        coefs[site].update(bookmark_info)

In [52]:
def format_annotations(sites, output_path=None):
    sites_df = pd.DataFrame(sites)
    if output_path:
        sites_df.to_csv(output_path, sep='\t', index=False)
    return sites_df

### Site Annotation and Export

In [34]:
# import reference annotations

hg38_refgene_annotations = {}

with io.BufferedReader(gzip.open(hg38_gtf, 'rb')) as ref:
    for b_line in tqdm(ref):
        line = b_line.decode()
        line_info = line.strip().split('\t')
        chrom, _, feature_type, start, end, score, strand, frame = line_info[0:-1]
        transcript_info = {}
        for info in line_info[-1].split(';'):
            seg = info.strip().split(' ')
            if len(seg)== 2:
                transcript_info[seg[0]] = seg[1].replace('"', '')
        info = dict(chrom=chrom, feature=feature_type, start=int(start), end=int(end), strand=strand, frame=frame, attributes=transcript_info)
        if transcript_info['gene_name'] not in hg38_refgene_annotations:
            hg38_refgene_annotations[transcript_info['gene_name']] = [info]
        else:
            hg38_refgene_annotations[transcript_info['gene_name']].append(info)

In [35]:
feature_spans = defaultdict(list)

count = 0

for feature, feature_info in tqdm(hg38_refgene_annotations.items()):
    transcript_info = feature_info[0]
    if transcript_info['feature'] != 'transcript':
        continue
    tss_start = transcript_info['start'] if transcript_info['strand'] == '+' else transcript_info['end']
    feature_spans[transcript_info['chrom']].append((tss_start, transcript_info['attributes']['gene_name']))

In [36]:
for feature_list in feature_spans.values():
    feature_list.sort(key=lambda x: x[0])

In [53]:
for bed_file in tqdm(bed_files):
        bed_info = []
        with open(bed_file, 'r') as bed:
            for line in bed:
                chrom, start, end = line.strip().split('\t')
                start, end = int(start), int(end)
                info = dict(chrom=chrom, start=start, end=end)
                info.update(binary_search((chrom, int(start)), feature_spans))
                info['hit_type'] = get_book_mark_info((chrom, int(start)), hg38_refgene_annotations[info['bookmark_gene']])
                bed_info.append(info)
        _ = format_annotations(bed_info, output_path=bed_file.replace('.bed', '.anno.tsv'))